In [2]:
import numpy as np
import pandas as pd
import pickle
import os
import json
import random
import tensorflow as tf
from tensorflow.keras.layers import *

In [4]:
import nltk
nltk.download("wordnet")
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Niki\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Niki\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
with open("intents.json") as intents_file:
    intents = json.load(intents_file)

In [6]:
print (intents)

{'intents': [{'tag': 'greetings', 'patterns': ['hello', 'hey', 'hi', 'good day', 'greetings', "what's up?", 'how is it going'], 'responses': ['hello', 'hey!', 'what can i do for you?']}, {'tag': 'goodbye', 'patterns': ['cya', 'see you later', 'goodbye', 'have a good day', 'bye', 'cao', 'see ya'], 'responses': ['have a nice day', 'goodbye']}, {'tag': 'age', 'patterns': ['how old', 'how old are you?', 'what is your age', 'how old are you', 'age?'], 'responses': ['I get reborn after every compilation', 'hey!', 'my owners are averagely 20 years!']}, {'tag': 'name', 'patterns': ['what is your name', 'what should i call you', "what's your name?", 'who are you?', 'can you tell me your name'], 'responses': ['you can call me Medbot!', 'i am Medbot!', 'i am Medbot your medical assistant']}, {'tag': 'common cold symptoms', 'patterns': ['Runny or stuffy nose', 'Sore throat', 'Cough', 'Congestion', 'Slight body aches or a mild headache', 'Sneezing', 'Low-grade fever', 'Generally feeling unwell (mal

In [13]:
words = []
classes = []
docs = []
ignore_case = [".", ",", "?", "!"]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        lst = nltk.word_tokenize(pattern)
        words.extend(lst)
        docs.append((lst, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])


In [15]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_case]
words = sorted(set(words))
classes = sorted((set(classes)))

In [16]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [18]:
training = []
output = [0]*len(classes)

for doc in docs:
    bag = []
    words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_case]
    word_pattern = doc[0]
    
    for word in words :
        bag.append(1) if word in word_pattern else bag.append(0)
    
    output_row = list(output)
    output_row[classes.index(doc[1])] = 1
    
    print(bag)
    print(output_row)
    
    training.append([bag, output_row])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [32]:
random.shuffle(training)
training = np.array(training)
X_train = list(training[:,0])
Y_train = list(training[:,1])

In [33]:
input_length = (len(X_train[0]),)
output_length = len(Y_train[0])

model = tf.keras.models.Sequential()

model.add(Dense(units = 128,
               input_shape = input_length,
               activation = "relu"))

model.add(Dropout(0.5))

model.add(Dense(units = 64,
               activation = "relu"))

model.add(Dropout(0.5))

model.add(Dense(units = output_length,
               activation = "softmax"))


In [ ]:
sgd = tf.keras.optimizers.SGD(lr = 0.001, weight_decay = 1e-6, momentum = 0.9, nesterov = True)

model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

history = model.fit(np.array(X_train), np.array(Y_train), epochs=200, batch_size=5,verbose=1)

model.save('chatbotmodel.h5', history)

print('Training Done')

Epoch 1/200
17/17 [==============================] - 1s 6ms/step - loss: 0.0358 - accuracy: 1.0000
Epoch 2/200
17/17 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.9762
Epoch 3/200
17/17 [==============================] - 0s 7ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 4/200
17/17 [==============================] - 0s 7ms/step - loss: 0.0366 - accuracy: 0.9881
Epoch 5/200
17/17 [==============================] - 0s 7ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 6/200
17/17 [==============================] - 0s 6ms/step - loss: 0.0350 - accuracy: 0.9881
Epoch 7/200
17/17 [==============================] - 0s 7ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 8/200
17/17 [==============================] - 0s 6ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 9/200
17/17 [==============================] - 0s 6ms/step - loss: 0.0350 - accuracy: 0.9881
Epoch 10/200
17/17 [==============================] - 0s 6ms/step - loss: 0.0531 - accuracy: 0.9881
Epoch 11/